## 1. Критерий информативности с нуля

как мы разобрали, построение дерева зависит от следующих факторов:
* вид правила разбиения
* критерий информативности
* критерий останова
* метод стрижки
* проблема пропусков

пройдёмся критериям информативности

In [2]:
import numpy as np
from collections import Counter

*Нам понадобятся две библиотеки: numpy вы знаете, а объект класса Counter в заданном списке просто подсчитывает количество вхождений каждого элемента и возвращает результат в виде словаря. Пример:*

In [5]:
t = Counter([9,9,9,7,7])
t.values()

dict_values([3, 2])

Ниже представлена функция для расчёта энтропийного критерия качества:

$H(R) = -\sum_{k=1}^{K}p_klogp_k$

In [12]:
l = [1, 1, 2, 1, 2, 3]
length = len(l)
length

6

In [13]:
cnt = Counter(l).values()
cnt

dict_values([3, 2, 1])

In [16]:
ps = [v / length for v in cnt]
ps

[0.5, 0.3333333333333333, 0.16666666666666666]

In [19]:
HR = -sum([p * np.log(p) for p in ps])
HR

1.0114042647073516

In [42]:
del(l)
l = [1, 2, 1, 2, 1, 1, 3, 3, 3]

In [43]:
def HEntropy(l):
    lenght = len(l)
    cnt = Counter(l).values()
    ps = [v / length for v in cnt]
    H = -sum([p * np.log(p) for p in ps])
    return H

In [44]:
HEntropy(l)

0.9830877585747855

**Задание.** По аналогии с *энтропийным критерием* заполните функции *критерия Джини*

In [45]:
def HGini(l):
    lenght = len(l)
    cnt = Counter(l).values()
    ps = [v / length for v in cnt]
    H = sum([p * (1 - p) for p in ps])
    return H

In [46]:
HGini(l)

0.6944444444444444

In [49]:
def IG(H, l, i):
    length = len(l)
    return H(l) - i / length * H(l[:i]) - (length - i) / length * H (l[i:])

In [51]:
IG(HGini, l, 5)

0.25925925925925924

In [52]:
def test_H(H, l):
    print("{:5} {:3}   {:4} {:4} {:4}".format("#","l","IG","Hl","Hr"))
    print("-"*24)
    for i in range(1,len(l)):
        print("{:2}. {:3}   {:.2f} {:.2f} {:.2f}".format(i, l[i], IG(H, l, i), H(l[:i]), H(l[i:])))

In [53]:
l = [1]*5 + [2]*3 + [1]*4
print(l)

[1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1]


In [54]:
test_H(HEntropy, l)

#     l     IG   Hl   Hr  
------------------------
 1.   1   -0.25 0.30 -0.04
 2.   1   -0.46 0.37 0.17
 3.   1   -0.61 0.35 0.35
 4.   1   -0.68 0.27 0.50
 5.   2   -0.68 0.15 0.62
 6.   2   -0.81 0.45 0.64
 7.   2   -0.80 0.52 0.57
 8.   1   -0.68 0.50 0.27
 9.   1   -0.61 0.35 0.35
10.   1   -0.46 0.17 0.37
11.   1   -0.25 -0.04 0.30


In [56]:
test_H(HGini, l[5:])

#     l     IG   Hl   Hr  
------------------------
 1.   2   0.07 0.14 0.44
 2.   2   0.15 0.22 0.36
 3.   1   0.24 0.25 0.22
 4.   1   0.14 0.39 0.25
 5.   1   0.07 0.47 0.22
 6.   1   0.02 0.50 0.14


### 2. Визуализация принятия решений классификатором sklearn

1. Используем данные о цветках ириса

In [57]:
from sklearn.datasets import load_iris
import pandas as pd

In [59]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns

In [61]:
iris = load_iris()